<a href="https://colab.research.google.com/github/janaabunasser14/ENCS5141-INTELLIGENT-SYSTEMS-LAB/blob/main/1201110_Experiment_9_Introduction_to_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.2 Procedure

In [3]:
pip install -U datasets transformers[torch] evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl

In [4]:
from datasets import load_dataset
dataset = load_dataset('MrbBakh/Sentiment140')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

## 1.2.1 Text Pre-processing

In [5]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

def tokenize(row):
    tokens = word_tokenize(row['text'])
    # Convert tokens to lowercase and remove punctuation
    tokens = [token.lower() for token in tokens if token.isalpha()]
    return {'tokens': tokens}

dataset = dataset.map(tokenize)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [6]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def remove_stopwords(row):
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in row['tokens'] if token not in stop_words]
    return {'tokens': tokens}

dataset = dataset.map(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Parameter 'function'=<function remove_stopwords at 0x7ca258533760> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

**Task 1: Use PorterStemmer from NLTK to stem the tokens.**

In [7]:
from nltk.stem import PorterStemmer
import nltk

nltk.download('punkt')

porter = PorterStemmer()

def stem_tokens(row):
    stemmed_tokens = [porter.stem(token) for token in row['tokens']]
    return {'tokens': stemmed_tokens}

dataset = dataset.map(stem_tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

## 1.2.2 Word Embedding


In [8]:
from gensim.models import Word2Vec
word_embedding = Word2Vec(dataset['train']['tokens'], vector_size=100,
  min_count=1, window=5, sg=1, hs=0, negative=10)

In [9]:
word_embedding.save('w2v.model')
word_embedding = Word2Vec.load('w2v.model')

## 1.2.3 Average Vector

In [10]:
def filter_tokens(example):
    return {'tokens': [token for token in example['tokens'] if token in word_embedding.wv]}

def mean_vector(example):
    return {'mean': word_embedding.wv[example['tokens']].mean(axis=0)}

dataset = dataset.map(filter_tokens) \
                 .filter(lambda e: len(e['tokens']) > 0) \
                 .map(mean_vector)



Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/39924 [00:00<?, ? examples/s]

Map:   0%|          | 0/4963 [00:00<?, ? examples/s]

Map:   0%|          | 0/4966 [00:00<?, ? examples/s]

In [11]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
X = np.array(dataset['train']['mean'])
y = np.array(dataset['train']['sentiment'])
clf = GaussianNB()
clf.fit(X, y)

GaussianNB()

**Task 2: Compute the accuracy and the confusion matrix of the trained classifier on the test dataset.**


In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming you have already trained your classifier 'clf'
X_test = np.array(dataset['test']['mean'])
y_test = np.array(dataset['test']['sentiment'])

# Predict sentiment labels for the test data
y_pred = clf.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.6645187273459525
Confusion Matrix:
[[2054  516]
 [1150 1246]]


## 1.2.4 LSTM

In [13]:
def vectorize(example):
    return {'vectors': word_embedding.wv[example['tokens']]}

dataset = dataset.map(vectorize)


Map:   0%|          | 0/39924 [00:00<?, ? examples/s]

Map:   0%|          | 0/4963 [00:00<?, ? examples/s]

Map:   0%|          | 0/4966 [00:00<?, ? examples/s]

In [14]:
import torch
import torch.nn as nn
lstm = nn.LSTM(100, 200)
sequence = torch.tensor(dataset['train'][0]['vectors'])
out, _ = lstm(sequence)

In [15]:
import torch
import torch.nn as nn

lstm = nn.LSTM(100, 200, 2, batch_first=True)

batch = [torch.tensor(sequence) for sequence in dataset['train'][0:4]['vectors']]
padded_batch = nn.utils.rnn.pad_sequence(batch)

out, _ = lstm(padded_batch)


**Task 3: slice the output of the LSTM to get the last token’s output for every sample in the batch.**

In [16]:
# Assuming 'out' is the output of the LSTM
# Extract the output of the last time step for each sequence in the batch
last_token_output = out[:, -1, :]

In [17]:
def word_to_index(example):
    indices = [word_embedding.wv.key_to_index[token] for token in example['tokens']]
    return {'indices': indices}

dataset = dataset.map(word_to_index)


Map:   0%|          | 0/39924 [00:00<?, ? examples/s]

Map:   0%|          | 0/4963 [00:00<?, ? examples/s]

Map:   0%|          | 0/4966 [00:00<?, ? examples/s]

In [18]:
import numpy as np

pad_vector = np.zeros(word_embedding.vector_size)
weights = np.vstack([word_embedding.wv.vectors, pad_vector])

vocab_size, embedding_size = weights.shape
pad_idx = vocab_size - 1


In [19]:
import torch
import torch.nn as nn

def pad_sequences(batch):
    indices = [torch.tensor(sample, dtype=torch.long) for sample in batch['indices']]
    indices = nn.utils.rnn.pad_sequence(indices, batch_first=True, padding_value=pad_idx)
    return {'indices': indices}

dataset = dataset.map(pad_sequences, batched=True, batch_size=None).with_format('torch')


Map:   0%|          | 0/39924 [00:00<?, ? examples/s]

Map:   0%|          | 0/4963 [00:00<?, ? examples/s]

Map:   0%|          | 0/4966 [00:00<?, ? examples/s]

In [20]:
import torch.nn.functional as F

class SentimentClassifierLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        embeddings = self.embedding(x)
        out, _ = self.lstm(embeddings)
        out = out[:, -1, :]
        out = self.fc(out)
        out = F.sigmoid(out)
        return out.squeeze(1)


In [21]:
hidden_size = 128
num_layers = 2

model = SentimentClassifierLSTM(vocab_size=vocab_size,
                                embedding_size=embedding_size,
                                hidden_size=hidden_size,
                                num_layers=num_layers)

model.embedding.weight = nn.Parameter(torch.FloatTensor(weights))
model.embedding.weight.requires_grad = False


In [22]:
learning_rate = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)


In [24]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 2048

def to_dataloader(dataset, split, shuffle):
    dataset = TensorDataset(dataset[split]['indices'], dataset[split]['sentiment'])
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_dataloader = to_dataloader(dataset, 'train', True)
test_dataloader = to_dataloader(dataset, 'test', False)
validation_dataloader = to_dataloader(dataset, 'validation', False)


In [25]:
def train_one_epoch(dataloader):
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


**Task 4: Use the train one epoch to train the model on 20 epochs. Bonus: Evaluate the model on the validation set after each epoch and print the validation accuracy**

In [26]:
num_epochs = 20

for epoch in range(num_epochs):
    # Train one epoch
    train_one_epoch(train_dataloader)

    # Evaluate on validation set
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        for inputs, labels in validation_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()
            outputs = model(inputs)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate validation accuracy
        validation_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {validation_accuracy:.4f}')

    model.train()


Epoch [1/20], Validation Accuracy: 0.5035
Epoch [2/20], Validation Accuracy: 0.6607
Epoch [3/20], Validation Accuracy: 0.6984
Epoch [4/20], Validation Accuracy: 0.7139
Epoch [5/20], Validation Accuracy: 0.7203
Epoch [6/20], Validation Accuracy: 0.7189
Epoch [7/20], Validation Accuracy: 0.7223
Epoch [8/20], Validation Accuracy: 0.7151
Epoch [9/20], Validation Accuracy: 0.7236
Epoch [10/20], Validation Accuracy: 0.7042
Epoch [11/20], Validation Accuracy: 0.7310
Epoch [12/20], Validation Accuracy: 0.7034
Epoch [13/20], Validation Accuracy: 0.7318
Epoch [14/20], Validation Accuracy: 0.7364
Epoch [15/20], Validation Accuracy: 0.7334
Epoch [16/20], Validation Accuracy: 0.7272
Epoch [17/20], Validation Accuracy: 0.7338
Epoch [18/20], Validation Accuracy: 0.7346
Epoch [19/20], Validation Accuracy: 0.7320
Epoch [20/20], Validation Accuracy: 0.7234


**Task 5: Evaluate the model on the test set using the accuracy and confusion matrix.**


In [27]:
# Evaluate on test set
with torch.no_grad():
    model.eval()
    test_correct = 0
    test_total = 0
    test_predictions = []
    test_labels = []
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        outputs = model(inputs)
        predicted = (outputs >= 0.5).float()
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

    # Calculate test accuracy
    test_accuracy = test_correct / test_total
    print(f'Test Accuracy: {test_accuracy:.4f}')

    # Calculate confusion matrix
    from sklearn.metrics import confusion_matrix
    test_predictions = np.array(test_predictions)
    test_labels = np.array(test_labels)
    test_conf_matrix = confusion_matrix(test_labels, test_predictions)
    print("Confusion Matrix:")
    print(test_conf_matrix)


Test Accuracy: 0.7292
Confusion Matrix:
[[1607  963]
 [ 382 2014]]


Task 6: Compare the performance of the model with the performance of the Average
Vector model

- LSTM Model: Like understanding a story by reading it word by word, understanding context and relationships. More accurate but takes longer to learn.

- Average Vector Model: Like getting a general idea by averaging key points. Simpler and faster but less accurate and doesn't understand context well.

In short, the LSTM model is like a detailed reader, while the Average Vector model is like a quick skimmer. The LSTM model is more accurate but slower to learn, whereas the Average Vector model is faster but less accurate. It depends on the trade-off between accuracy and speed.








## 1.2.5 Transformers

In [28]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('lyeonii/bert-mini')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [29]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x['text'],
        padding=True,
        return_tensors='pt'
    ),
    batched=True,
    batch_size=None
).with_format('torch')


Map:   0%|          | 0/39924 [00:00<?, ? examples/s]

Map:   0%|          | 0/4963 [00:00<?, ? examples/s]

Map:   0%|          | 0/4966 [00:00<?, ? examples/s]

In [30]:
tokenized_dataset = tokenized_dataset.rename_column('sentiment', 'labels')

In [31]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('lyeonii/bert-mini', num_labels=2)


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lyeonii/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Task 7: Use compute metrics in Trainer constructor, with evaluate package, to
compute validation accuracy

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='sentiment-analysis',
    num_train_epochs=3,
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation']
)

trainer.train()

**Task 8: Evaluate the model on the test set using the accuracy and confusion matrix**

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Evaluate on test set
eval_results = trainer.evaluate(tokenized_dataset['test'])

# Extract predictions and labels
predictions = np.argmax(eval_results.predictions, axis=1)
labels = eval_results.label_ids

# Calculate accuracy
accuracy = np.mean(predictions == labels)
print("Test Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)


**Task 9: Compare the model with the previously trained models.**

In comparison to previously trained models like LSTM and Average Vector models, the BERT-based model for sentiment analysis demonstrates superior performance. BERT's transformer architecture enables it to capture intricate contextual information and complex language patterns effectively. Fine-tuning pre-trained BERT models for sentiment analysis tasks requires less data and time compared to training LSTM models from scratch. Consequently, BERT achieves higher accuracy and better performance in sentiment analysis tasks, making it the preferred choice for such applications.